In [54]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import pickle
import numpy as np
import time

# Carga los datos desde los archivos pickle
pickle_in = open("X.pickle", "rb")
X = np.array(pickle.load(pickle_in))

pickle_in = open("y.pickle", "rb")
y = np.array(pickle.load(pickle_in))

# Normaliza los datos
X = X / 255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(512))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            model.fit(X, y, batch_size=32, epochs=7, validation_split=0.3, callbacks=[tensorboard])
            
model.save('64x3-CNN.keras')  # Cambiado a .keras


3-conv-64-nodes-0-dense-1717842622
Epoch 1/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.5845 - loss: 0.6640 - val_accuracy: 0.7265 - val_loss: 0.5480
Epoch 2/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 15s 28ms/step - accuracy: 0.7439 - loss: 0.5218 - val_accuracy: 0.7602 - val_loss: 0.5067
Epoch 3/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.7786 - loss: 0.4706 - val_accuracy: 0.7750 - val_loss: 0.4733
Epoch 4/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 15s 28ms/step - accuracy: 0.8033 - loss: 0.4336 - val_accuracy: 0.7918 - val_loss: 0.4367
Epoch 5/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.8172 - loss: 0.3978 - val_accuracy: 0.8053 - val_loss: 0.4273
Epoch 6/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 15s 28ms/step - accuracy: 0.8420 - loss: 0.3542 - val_accuracy: 0.8002 - val_loss: 0.4167
Epoch 7/7
546/546 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 0.8604 - loss: 0.3286 - val_accuracy: 0.8001 - val_loss: 0.4211


In [60]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # usarás esto para convertir el número de predicción en un valor de cadena

def prepare(filepath):
    IMG_SIZE = 75  # Cambiamos el tamaño de la imagen a 75x75
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # Devolvemos la imagen con la forma (1, 75, 75, 1)

model = tf.keras.models.load_model("64x3-CNN.keras")
prediction = model.predict(prepare('cat.jpg'))  # No es necesario pasar una lista
print(prediction)

# Interpretar el resultado
print(CATEGORIES[int(prediction[0][0])])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[[1.1833292e-29]]
Dog


In [61]:
prediction = model.predict([prepare('cat.jpg')])
print(CATEGORIES[int(prediction[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Dog


In [62]:
prediction = model.predict([prepare('dog.jpg')])
print(CATEGORIES[int(prediction[0][0])])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Cat
